In [1]:
import numpy as np
import time
import pickle
from tensorflow.keras.datasets.mnist import load_data
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def data_reshape(x):
    x = (x.reshape(len(x), 28 * 28)).astype(np.float64)
    x /= x.max()
    return x

def model_save(model, filename):
    pickle.dump(model, open(filename, 'wb'))

def model_load():
    return pickle.load(open(filename, 'rb'))

def save_accuracy(accuracy, filename):
    with open(filename, mode='w') as f:
        for d in accuracy:
            f.write("%s\n" % d)
    
def save_times(times, filename):
    with open(filename, mode='w') as f:
        for d in times:
            f.write("%s\n" % d)

In [3]:
# MNISTデータの読み込み
(x, y), (xt, yt) = load_data()

In [4]:
x.shape

(60000, 28, 28)

In [5]:
len(x)

60000

In [6]:
xt.shape

(10000, 28, 28)

In [7]:
len(xt)

10000

In [8]:
# 画像データの正規化
x = data_reshape(x)
xt = data_reshape(xt)

In [18]:
# parameters----------------------------------

# 検証用データとして確保する割合
value_size = 0.01

# 訓練時のテストデータとして確保する割合
test_size = 0.1
random_state = 60

# --------------------------------------------

# 何%を検証用にするか
nVals = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
# nVals = np.array([1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])
print(nVals)
print("trainData: ", int((1-nVals[2]) * len(x)))

# trainData, valData, trainLabels, valLabels = train_test_split(x, y,
#                                                               test_size=1-nVals[0],
#                                                               random_state=random_state
#                                                               )
# print("testData: ", len(valData))

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
trainData:  42000


In [10]:
activation_function_list = ['tanh', 'relu', 'logistic']
for activation_function in activation_function_list:
    accuracies = []  # 正確度保存用
    proc_time = []   # 実行時間保存用
    for per in nVals:

        if per == 1:
            trainData = x
            trainLabels = y
        else:
            trainData, valData, trainLabels, valLabels = train_test_split(x, y,
                                                                    test_size=1-per,
                                                                    random_state=random_state
                                                                   )

        start = time.time()  # 時間計測開始
        model = MLPClassifier(hidden_layer_sizes=(100,),
                              activation=activation_function,
                              max_iter=1000
                             )
        model.fit(trainData, trainLabels)
        score = model.score(xt, yt)
        duration = time.time() - start  # 計測終了

        accuracies.append(score)
        proc_time.append(duration)

        # モデルを保存する
        filename = 'train_data='+ str(len(trainData)) + '-activation_function=' + activation_function + '.sav'
        model_save(model, filename)

    filename = 'train_data='+ str(len(trainData)) + '-activation_function=' + activation_function + '-accuracy.sav'
    save_accuracy(accuracies, filename)
    filename = 'train_data='+ str(len(trainData)) + '-activation_function=' + activation_function + '-time.sav'
    save_times(proc_time, filename)

In [11]:
# # 保存したモデルをロードする
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)